# Financial Research Assistant

This notebook demonstrates how to build a multi-agent collaboration system for financial research using Amazon Bedrock. The system uses a supervisor agent pattern to orchestrate specialized sub-agents.

## What We'll Build

- **Supervisor Agent**: Coordinates the overall research process
- **News Agent**: Retrieves financial news and analyzes documents
- **Quantitative Analysis Agent**: Fetches stock data and optimizes portfolios
- **Smart Summarizer Agent**: Synthesizes insights into structured reports

> **Note**: Results from these agents should not be taken as financial advice.

## Prerequisites

Before running this notebook:
1. Complete the setup from the main README
2. Deploy the Web Search Lambda tool
3. Deploy the Stock Data Lambda tool
4. Enable required foundation models in Amazon Bedrock

## Step 1: Install Dependencies

Make sure all required packages are installed before proceeding.

In [ ]:
# Install required dependencies - run this first!
!pip install -q boto3 botocore opensearch-py pydantic rich

## Step 2: Import Required Modules

We'll import our helper classes that handle the complex Bedrock API interactions.

In [ ]:
import sys
import os
from pathlib import Path
from datetime import datetime
import time
import json

import boto3
from botocore.exceptions import ClientError
from IPython.display import display, clear_output

# Add the project root to the Python path so we can import our utilities
ROOT_PATH = Path.cwd().parents[2]
sys.path.insert(0, str(ROOT_PATH))

# Import our custom helper modules
from src.utils.bedrock_agent import (
    Agent,
    SupervisorAgent,
    Task,
    Guardrail,
    region,
    account_id,
    agents_helper,
)
from src.utils.knowledge_base_helper import KnowledgeBasesForAmazonBedrock

# Initialize helpers
kb_helper = KnowledgeBasesForAmazonBedrock()
bedrock_client = boto3.client("bedrock")

# Configure the LLM model to use for agents
# You can switch between different Claude versions or Amazon models
LLM = "us.anthropic.claude-3-5-sonnet-20241022-v2:0"

print(f"AWS Account: {account_id}")
print(f"Region: {region}")
print(f"Using LLM: {LLM}")

## Step 3: Set Up S3 Buckets

We need buckets for:
- **Knowledge Base Data**: Store financial documents (10-K reports, earnings calls, etc.)
- **BDA Processing**: Bedrock Data Automation input/output

In [ ]:
# Initialize S3 client
s3_client = boto3.client("s3", region_name=region)

# Create bucket names with account and region for uniqueness
kb_bucket_name = f"financial-research-data-{region}-{account_id}"

def create_bucket_if_not_exists(bucket_name: str):
    """Create an S3 bucket if it doesn't already exist."""
    try:
        if region == "us-east-1":
            # us-east-1 doesn't need LocationConstraint
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={"LocationConstraint": region}
            )
        print(f"✓ Created bucket: {bucket_name}")
    except ClientError as e:
        if e.response["Error"]["Code"] in ["BucketAlreadyOwnedByYou"]:
            print(f"✓ Bucket already exists: {bucket_name}")
        else:
            raise

# Create the knowledge base bucket
create_bucket_if_not_exists(kb_bucket_name)

## Step 4: Upload Sample Financial Documents (Optional)

If you have financial documents to analyze, upload them to the knowledge base bucket.
This step is optional - the agents can still function using web search.

In [ ]:
# Optional: Download sample financial documents
# Uncomment the lines below to download Amazon financial reports

# !mkdir -p files/
# !curl -L -o files/amazon2024_10k.pdf https://s2.q4cdn.com/299287126/files/doc_financials/2024/q4/e42c2068-bad5-4ab6-ae57-36ff8b2aeffd.pdf

# Upload to S3 if you have local documents
# import os
# for file in os.listdir("files/"):
#     local_path = f"files/{file}"
#     s3_key = f"data/{file}"
#     print(f"Uploading {file} to s3://{kb_bucket_name}/{s3_key}")
#     s3_client.upload_file(local_path, kb_bucket_name, s3_key)

print("Sample data step complete (skipped by default)")

## Step 5: Define Cleanup Functions

These functions help manage the lifecycle of our agents and guardrails.

In [ ]:
def clean_up_agents():
    """
    Delete all agents and guardrails created by this notebook.
    
    Call this function to clean up resources when you're done
    experimenting or if you want to recreate agents from scratch.
    """
    agent_names = [
        "financial_research_assistant",
        "news_agent",
        "quantitative_analysis_agent",
        "smart_summarizer_agent",
    ]
    
    print("Cleaning up agents...")
    for name in agent_names:
        try:
            agents_helper.delete_agent(
                agent_name=name,
                delete_role_flag=True,
                verbose=True
            )
        except Exception as e:
            print(f"Could not delete {name}: {e}")
    
    # Also clean up any guardrails we created
    print("Cleaning up guardrails...")
    try:
        response = bedrock_client.list_guardrails()
        for guardrail in response.get("guardrails", []):
            if guardrail["name"] == "no_crypto_guardrail":
                print(f"Deleting guardrail: {guardrail['id']}")
                bedrock_client.delete_guardrail(guardrailIdentifier=guardrail["id"])
    except Exception as e:
        print(f"Could not clean up guardrails: {e}")
    
    print("Cleanup complete!")

# Uncomment the line below to clean up existing agents before creating new ones
# clean_up_agents()

## Step 6: Create a Guardrail

Guardrails help ensure responsible AI usage. In this example, we create a
guardrail that prevents discussion of cryptocurrency topics.

In [ ]:
def create_guardrail():
    """
    Create a guardrail to filter out cryptocurrency discussions.
    
    This is a safety measure to keep the agents focused on
    traditional financial analysis.
    """
    return Guardrail(
        name="no_crypto_guardrail",
        topic_name="cryptocurrency_topic",
        description="Prevents discussion of cryptocurrency and blockchain investments.",
        denied_topics=["bitcoin", "crypto", "cryptocurrency", "ethereum", "blockchain"],
        blocked_input_response="I apologize, but I cannot provide analysis on cryptocurrency investments.",
        verbose=True
    )

# Force recreation of agents for a clean slate
Agent.set_force_recreate_default(True)

# Create the guardrail (optional - uncomment to enable)
# no_crypto_guardrail = create_guardrail()
no_crypto_guardrail = None  # Set to None to skip guardrail

print("Guardrail setup complete")

## Step 7: Create the Smart Summarizer Agent

This agent specializes in synthesizing information from other agents
into structured investment insights.

In [ ]:
# The Smart Summarizer takes output from other agents and creates
# coherent, structured summaries for investment decisions

smart_summarizer_agent = Agent.create(
    name="smart_summarizer_agent",
    role="Financial Analyst specializing in investment insight synthesis",
    goal="Analyze stock trends and market news to generate structured investment insights.",
    instructions="""You are a Financial Analyst responsible for synthesizing stock trends 
and financial news into structured insights.

Your responsibilities:
- Combine stock price trends with financial news to identify key patterns
- Analyze macroeconomic indicators, company earnings, and market sentiment
- Ensure responses are fact-driven, clearly structured, and cite sources when possible
- Keep analyses concise and focused on major trends and anomalies

Important guidelines:
- Do NOT provide financial advice - analyze and summarize data objectively
- If given portfolio optimization percentages, note they are mathematical results, not advice
- Maintain a professional tone without using emojis
- Structure your output with clear sections and bullet points""",
    llm=LLM,
)

print(f"✓ Created Smart Summarizer Agent (ID: {smart_summarizer_agent.agent_id})")

## Step 8: Create the Quantitative Analysis Agent

This agent handles stock data retrieval and portfolio optimization.
It uses Lambda tools for real-time market data access.

In [ ]:
# Lambda ARN for stock data tools - update this after deployment
stock_data_tools_arn = f"arn:aws:lambda:{region}:{account_id}:function:stock_data_tools"

quantitative_analysis_agent = Agent.create(
    name="quantitative_analysis_agent",
    role="Stock Data and Portfolio Optimization Specialist",
    goal="Retrieve real-time and historical stock prices, and optimize investment portfolios.",
    instructions="""You are a Stock Data and Portfolio Optimization Specialist.

Your capabilities:
1. Retrieve stock price data using the stock_data_lookup tool
2. Perform portfolio optimization with at least 3 stock tickers
3. Analyze historical price movements and volatility

Core behaviors:
- Always retrieve stock data first before running portfolio optimization
- If fewer than 3 tickers are provided for optimization, inform the user
- Focus on data retrieval and mathematical analysis - do not interpret trends
- Return data in a clear, structured format""",
    tools=[
        # Stock Data Lookup Tool
        {
            "code": stock_data_tools_arn,
            "definition": {
                "name": "stock_data_lookup",
                "description": "Gets the 1-month stock price history for a given ticker, formatted as JSON.",
                "parameters": {
                    "ticker": {
                        "description": "The stock ticker symbol to retrieve price history for (e.g., AAPL)",
                        "type": "string",
                        "required": True,
                    }
                },
            },
        },
        # Portfolio Optimization Tool
        {
            "code": stock_data_tools_arn,
            "definition": {
                "name": "portfolio_optimization",
                "description": "Optimizes a stock portfolio given a list of tickers. Requires at least 3 tickers.",
                "parameters": {
                    "tickers": {
                        "description": "A comma-separated list of stock tickers (minimum 3)",
                        "type": "string",
                        "required": True,
                    },
                    "prices": {
                        "description": "JSON object with historical prices from stock_data_lookup",
                        "type": "string",
                        "required": True,
                    },
                },
            },
        },
    ],
    llm=LLM,
)

print(f"✓ Created Quantitative Analysis Agent (ID: {quantitative_analysis_agent.agent_id})")

## Step 9: Create the News Agent with Knowledge Base

This agent retrieves financial news and can search a knowledge base
containing SEC filings, earnings calls, and other financial documents.

In [ ]:
# Create or retrieve the knowledge base for financial documents
kb_name = "financial-research-kb"
kb_description = "Knowledge base for financial analysis containing 10-K reports, earnings calls, and SEC filings."

# This creates a new KB or retrieves an existing one
kb_id, ds_id = kb_helper.create_or_retrieve_knowledge_base(
    kb_name=kb_name,
    kb_description=kb_description,
    data_bucket_name=kb_bucket_name,
    embedding_model="amazon.titan-embed-text-v2:0",
)

print(f"✓ Knowledge Base ID: {kb_id}")
print(f"✓ Data Source ID: {ds_id}")

In [ ]:
# Lambda ARN for web search - update this after deployment
web_search_arn = f"arn:aws:lambda:{region}:{account_id}:function:web_search"

news_agent = Agent.create(
    name="news_agent",
    role="Market News Researcher and Financial Document Analyst",
    goal="Retrieve insights from the knowledge base and fetch latest financial news when needed.",
    instructions=f"""You are a Financial Document & News Analyst responsible for extracting 
insights from official financial reports and real-time news.

Your capabilities:
1. Extract insights from earnings calls, SEC filings, and press releases in the knowledge base (ID: {kb_id})
2. Summarize financial reports with focus on factual accuracy
3. Retrieve latest financial news when the knowledge base lacks relevant information

Core behaviors:
- ALWAYS check the knowledge base (ID: {kb_id}) FIRST before fetching external news
- Avoid unnecessary web searches - use external sources only when KB is insufficient
- Ensure all findings are fact-based, neutral, and structured for investment research
- Cite sources clearly in your responses""",
    tools=[
        {
            "code": web_search_arn,
            "definition": {
                "name": "web_search",
                "description": "Searches the web for financial news, earnings reports, and market updates.",
                "parameters": {
                    "search_query": {
                        "description": "The query to search the web with",
                        "type": "string",
                        "required": True,
                    },
                    "target_website": {
                        "description": "Specific website to search (e.g., reuters.com)",
                        "type": "string",
                        "required": False,
                    },
                    "topic": {
                        "description": "Topic category (e.g., 'news', 'finance')",
                        "type": "string",
                        "required": False,
                    },
                    "days": {
                        "description": "Number of days of history to search",
                        "type": "string",
                        "required": False,
                    },
                },
            },
        },
    ],
    kb_id=kb_id,
    llm=LLM,
)

print(f"✓ Created News Agent (ID: {news_agent.agent_id})")

In [ ]:
# Synchronize the knowledge base if you uploaded documents
# This step is optional if you don't have any documents to sync

# Uncomment to sync the knowledge base:
# print("Synchronizing knowledge base...")
# kb_helper.synchronize_data(kb_id, ds_id)

print("Knowledge base synchronization step complete (skipped by default)")

## Step 10: Create the Supervisor Agent

The supervisor agent orchestrates all sub-agents to produce comprehensive
investment research by delegating tasks and consolidating results.

In [ ]:
financial_research_assistant = SupervisorAgent.create(
    name="financial_research_assistant",
    role="Investment Research Assistant and Multi-Agent Coordinator",
    goal="Orchestrate specialized agents to conduct comprehensive stock analysis and produce structured investment reports.",
    instructions=f"""You are an Investment Research Assistant responsible for coordinating 
financial research from specialized agents and producing structured insights.

Your capabilities:
1. Manage collaboration between sub-agents to retrieve and analyze financial data
2. Synthesize stock trends, financial reports, and market news into structured analysis
3. Deliver well-organized, fact-based investment insights with clear source attribution

Available sub-agents:
- **news_agent**: Retrieves and summarizes financial news and documents
  - Always instruct it to check the knowledge base (ID: {kb_id}) first
- **quantitative_analysis_agent**: Provides stock prices and portfolio optimization
  - For optimization, requires at least 3 tickers
- **smart_summarizer_agent**: Synthesizes data into structured investment insights

Core behaviors:
- Only invoke a sub-agent when necessary for the user's request
- Ensure responses are well-structured and relevant to investment decisions
- Clearly distinguish between news analysis, technical data, and synthesized insights
- Never provide specific investment recommendations - present analysis objectively""",
    collaboration_type="SUPERVISOR",
    collaborator_agents=[
        {
            "agent": "news_agent",
            "instructions": f"Always check the knowledge base (ID: {kb_id}) first. Use for financial news and document analysis.",
        },
        {
            "agent": "quantitative_analysis_agent",
            "instructions": "Use for retrieving stock price history and performing portfolio optimization.",
        },
        {
            "agent": "smart_summarizer_agent",
            "instructions": "Use for synthesizing data and generating structured investment insights.",
        },
    ],
    collaborator_objects=[
        news_agent,
        quantitative_analysis_agent,
        smart_summarizer_agent,
    ],
    guardrail=no_crypto_guardrail,
    llm=LLM,
)

print(f"✓ Created Financial Research Assistant Supervisor (ID: {financial_research_assistant.agent_id})")
print("")
print("All agents are ready! You can now test the system with example queries.")

## Step 11: Test the Multi-Agent System

Now let's test our financial research assistant with some example queries.
The supervisor will coordinate the sub-agents to answer each question.

In [ ]:
# Example 1: Stock price analysis with news correlation
# This will use the quantitative agent for prices and news agent for context

request = "What's AAPL stock price doing over the last week and relate that to recent news?"

print(f"Request: {request}")
print("-" * 50)

result = financial_research_assistant.invoke(
    request,
    enable_trace=False,
    trace_level="core"
)

print(f"\nResponse:\n{result}")

In [ ]:
# Example 2: Portfolio optimization
# This demonstrates the quantitative agent's optimization capability

request = "Optimize my portfolio with AAPL, MSFT, and GOOGL. Show me the recommended allocations."

print(f"Request: {request}")
print("-" * 50)

result = financial_research_assistant.invoke(
    request,
    enable_trace=False,
    trace_level="core"
)

print(f"\nResponse:\n{result}")

In [ ]:
# Example 3: Custom query
# Try your own financial research question!

# Uncomment and modify the request below:
# request = "Your custom financial research question here"
request = "Compare the recent performance of tech giants: MSFT, GOOGL, and META"

print(f"Request: {request}")
print("-" * 50)

result = financial_research_assistant.invoke(
    request,
    enable_trace=False,
    trace_level="core"
)

print(f"\nResponse:\n{result}")

## Step 12: Cleanup (Optional)

When you're done experimenting, run this cell to delete all created resources.
This helps avoid ongoing charges for the deployed agents.

In [ ]:
# Uncomment the line below to clean up all resources
# clean_up_agents()

# To also delete the Lambda tools, run:
# !cd ../../../src/shared/web_search && sam delete --stack-name web-search-stack
# !cd ../../../src/shared/stock_data && sam delete --stack-name stock-data-stack

print("Cleanup instructions complete. Uncomment the lines above to delete resources.")